In [41]:
df = spark.sql("SELECT * FROM bing_lake_db.raw.tbl_latest_news")
display(df)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 43, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 38b631b1-3f1e-469f-9211-9ec3868ca744)

In [42]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 44, Finished, Available, Finished, False)

In [43]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("snippet")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error") #no error->null
)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 45, Finished, Available, Finished, False)

In [44]:
#Apply the model to our dataframe
result = model.transform(df)


StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 46, Finished, Available, Finished, False)

In [45]:
display(result)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 47, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 0e98e524-f473-4ed9-a2a0-7d632e3340fd)

In [46]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 48, Finished, Available, Finished, False)

In [47]:
display(sentiment_df)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 49, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, ad0aa8ed-5c7d-435f-9304-7e7b6dde106b)

In [48]:
sentiment_df_final = sentiment_df.drop("error", "response")
display(sentiment_df_final)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 50, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 5a5a2fda-5710-4ab8-83c9-9b21cb7eeb43)

In [49]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.raw.tbl_sentiment_analysis'
    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.link = target_table.link

                    WHEN MATCHED AND
                    source_view.title <> target_table.title OR
                    source_view.snippet <> target_table.snippet OR
                    source_view.source <> target_table.source OR
                    source_view.thumbnail <> target_table.thumbnail 

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 51, Finished, Available, Finished, False)

Table Already Exists


In [50]:
"""
from pyspark.functions import col, to_date

df = df.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy")

df.printSchema()

df.write.format('delta').mode("overwrite").option("overwriteSchema", "True").saveAsTable(table_name)

"""

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 52, Finished, Available, Finished, False)

'\nfrom pyspark.functions import col, to_date\n\ndf = df.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy")\n\ndf.printSchema()\n\ndf.write.format(\'delta\').mode("overwrite").option("overwriteSchema", "True").saveAsTable(table_name)\n\n'

In [51]:


spark.sql("""
CREATE TABLE IF NOT EXISTS bing_lake_db.raw.daily_sentiment_alerts (
  alert_date DATE,
  metric_name STRING,
  metric_value DOUBLE,
  threshold DOUBLE,
  sent_at TIMESTAMP
) USING DELTA
""")

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 53, Finished, Available, Finished, False)

DataFrame[]

In [52]:
import requests

WEBHOOK_URL = "https://default4fce493f17434d33bdec79e288863c.19.environment.api.powerplatform.com:443/powerautomate/automations/direct/workflows/7c3fe8e945934ca8824613c7ad95fdb9/triggers/manual/paths/invoke?api-version=1&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=yqBQeBDxXOnF2I2QjT8-39vyVGfIfEmW03EfvO1tRQ4"

payload = {"text": "🚨 Test alert from Fabric notebook"}
r = requests.post(WEBHOOK_URL, json=payload, timeout=30)
print(r.status_code, r.text[:300])

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 54, Finished, Available, Finished, False)

202 


In [76]:
from pyspark.sql.functions import col, current_date, current_timestamp, sum as Fsum, count as Fcount
from pyspark.sql.utils import AnalysisException
import requests

TABLE = "bing_lake_db.raw.tbl_latest_news"
ALERT_TABLE = "bing_lake_db.raw.daily_sentiment_alerts"
THRESHOLD = 0.30

WEBHOOK_URL = "https://default4fce493f17434d33bdec79e288863c.19.environment.api.powerplatform.com:443/powerautomate/automations/direct/workflows/7c3fe8e945934ca8824613c7ad95fdb9/triggers/manual/paths/invoke?api-version=1&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=yqBQeBDxXOnF2I2QjT8-39vyVGfIfEmW03EfvO1tRQ4"

df = sentiment_df_final

# 1) 今天資料（先用全部；之後要改 last_loaded_date 再加 filter）
df_today = df

# 2) 計算 positive 佔比
agg = (
    df_today
    .select(col("sentiment"))
    .groupBy()
    .agg(
        (Fsum((col("sentiment") == "positive").cast("int")) / Fcount("*")).alias("positive_ratio"),
        Fcount("*").alias("n_rows")
    )
)

row = agg.collect()[0]
n_rows = int(row["n_rows"])
positive_ratio = float(row["positive_ratio"]) if n_rows > 0 else 0.0

print("rows:", n_rows, "positive_ratio:", positive_ratio)

# 3) 確保 ALERT_TABLE 存在（不存在就建立空表）
try:
    spark.table(ALERT_TABLE).limit(1).count()
except AnalysisException:
    schema = "alert_date DATE, metric_name STRING, metric_value DOUBLE, threshold DOUBLE, sent_at TIMESTAMP"
    spark.createDataFrame([], schema).write.mode("overwrite").format("delta").saveAsTable(ALERT_TABLE)

# 4) 避免同一天重複通知
already_sent = (
    spark.table(ALERT_TABLE)
    .where((col("alert_date") == current_date()) & (col("metric_name") == "positive_ratio"))
    .limit(1)
    .count() > 0
)

print("already_sent:", already_sent)

# 5) 觸發條件
if  (positive_ratio >= THRESHOLD) :

    today_str = str(spark.sql("SELECT current_date()").collect()[0][0])

    # 給人看的文字（放到卡片內文）
    text = (
        f"Date: {today_str}\n"

        f"Positive ratio: {positive_ratio:.2%} (threshold {THRESHOLD:.0%})\n"

        f"Rows evaluated: {n_rows}\n"
        
        f"Workspace: Search Data Analytics"
    )

    # ✅ Adaptive Card payload（TeamsWebhook trigger 原生支援 attachments）
    card_payload = {
        "type": "message",
        "attachments": [
            {
                "contentType": "application/vnd.microsoft.card.adaptive",
                "content": {
                    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                    "type": "AdaptiveCard",
                    "version": "1.4",
                    "body": [
                        {
                            "type": "TextBlock",
                            "text": "🚨 Daily Sentiment Alert",
                            "weight": "Bolder",
                            "size": "Medium"
                        },
                        {
                            "type": "TextBlock",
                            "text": text,
                            "wrap": True
                        }
                    ]
                }
            }
        ]
    }

    resp = requests.post(WEBHOOK_URL, json=card_payload, timeout=30)
    print("Webhook:", resp.status_code, resp.text[:200])

    # 6) 記錄已通知（防重）
    spark.createDataFrame([(
        spark.sql("SELECT current_date()").collect()[0][0],
        "positive_ratio",
        positive_ratio,
        THRESHOLD,
        spark.sql("SELECT current_timestamp()").collect()[0][0]
    )], "alert_date DATE, metric_name STRING, metric_value DOUBLE, threshold DOUBLE, sent_at TIMESTAMP") \
    .write.mode("append").format("delta").saveAsTable(ALERT_TABLE)

else:
    print("No alert triggered (or already sent).")

StatementMeta(, 740767a1-cd3f-4474-a583-e15e66397404, 78, Finished, Available, Finished, False)

rows: 36 positive_ratio: 0.3055555555555556
already_sent: True
Webhook: 202 
